# Oil Well Project

## Preparing Data

In [1]:
# imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score,  roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, root_mean_squared_error
import plotly.express as px
from sklearn.neighbors import NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

All needed imports

In [2]:
data0 = pd.read_csv(r"C:\Users\alexi\Desktop\Coding Projects\Oil-Well-Project\Oil-well-Project\geo_data_0.csv")
data1 = pd.read_csv(r"C:\Users\alexi\Desktop\Coding Projects\Oil-Well-Project\Oil-well-Project\geo_data_1.csv")
data2 = pd.read_csv(r"C:\Users\alexi\Desktop\Coding Projects\Oil-Well-Project\Oil-well-Project\geo_data_2.csv")

CSV files

In [3]:
data0.info()
data1.info()
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [4]:
data0.duplicated().sum()

np.int64(0)

In [5]:
data1.duplicated().sum()

np.int64(0)

In [6]:
data2.duplicated().sum()

np.int64(0)

all 3 data frames are not missing any values or have any duplicate rows

## Train and Test Models

In [7]:
# Split data Into Training and Validation
data0_train, data0_val = train_test_split(data0, test_size=0.25, random_state=50)
data1_train, data1_val = train_test_split(data1, test_size=0.25, random_state=50)
data2_train, data2_val = train_test_split(data2, test_size=0.25, random_state=50)

# Print Results
{
    "data0": {"Train": data0_train.shape, "Val": data0_val.shape},
    "data1": {"Train": data1_train.shape, "Val": data1_val.shape},
    "data2": {"Train": data2_train.shape, "Val": data2_val.shape}
}


{'data0': {'Train': (75000, 5), 'Val': (25000, 5)},
 'data1': {'Train': (75000, 5), 'Val': (25000, 5)},
 'data2': {'Train': (75000, 5), 'Val': (25000, 5)}}

a 75/25 training/validation split on all 3 datasets

In [8]:
# Fucntion to train linear regression model
def train_and_predict_linear(train_df, val_df):
    x_train = train_df[['f0', 'f1', 'f2']]
    y_train = train_df[['product']]
    x_val = val_df[['f0', 'f1', 'f2']]
    y_val = val_df[['product']]

    model = LinearRegression()
    model.fit(x_train, y_train)
    predict = model.predict(x_val)
    mse = root_mean_squared_error(y_val, predict)

    return predict, mse

# Train and predict With Linear Regression
pred0, mse0 = train_and_predict_linear(data0_train, data0_val)
pred1, mse1 = train_and_predict_linear(data1_train, data1_val)
pred2, mse2 = train_and_predict_linear(data2_train, data2_val)

# Print Results
{
    "data0": {"mse": mse0},
    "data1": {"mse": mse1},
    "data2": {"mse": mse2} 
}

{'data0': {'mse': 37.596556043309086},
 'data1': {'mse': 0.8941004282530967},
 'data2': {'mse': 40.11354754639584}}

In [9]:
# Add predictions to the validation sets
data0_val_results = data0_val.copy()
data0_val_results['predicted_product'] = pred0

data1_val_results = data1_val.copy()
data1_val_results['predicted_product'] = pred1

data2_val_results = data2_val.copy()
data2_val_results['predicted_product'] = pred2


In [10]:
data0_val_results.head()

,id,f0,f1,f2,product,predicted_product
35363,9xDYu,-0.029148,0.933851,0.960092,97.109002,70.752036
43649,4ZV4Z,1.038490,-0.441222,1.267050,132.860001,96.184940
60832,7fGVJ,1.897318,-0.250727,1.956568,168.469937,101.249241
37399,qOp82,0.827823,-0.508357,3.058204,93.967682,108.201838
61034,DCX28,1.328177,-0.332757,-4.286053,106.358376,58.985978


In [11]:
data1_val_results.head()

,id,f0,f1,f2,product,predicted_product
35363,i20Ym,-11.833189,-2.478573,4.000121,110.992147,111.230660
43649,5EZpz,-14.822505,-11.935498,4.998859,137.945408,138.782825
60832,Bd1bL,-3.514141,-1.983008,2.002521,57.085625,56.176671
37399,YrgDZ,7.398975,-6.882181,2.000820,53.906522,54.653156
61034,XJHt1,7.447826,-6.047519,5.003255,134.766305,135.546890


In [12]:
data2_val_results.head()

,id,f0,f1,f2,product,predicted_product
35363,znjYt,0.071207,-0.477391,0.923855,71.624696,85.917109
43649,FmEYX,2.080727,-0.571323,-3.767611,61.753176,58.990207
60832,FBauH,-0.351986,-1.990903,1.019369,44.838522,86.412142
37399,JUiit,0.291070,-0.278772,7.977529,54.966447,126.423389
61034,G8Xz0,0.225914,1.396970,4.466413,35.103100,106.320373


I had all 3 datasets predict on linear regression and copied their results to 3 new data sets

In [13]:
# Average Volumes and RSME
results_summary = {
    "data0": {
        "average_predicted_volume": np.mean(pred0),
        "rsme": root_mean_squared_error(data0_val['product'], pred0)

    },
    "data1": {
        "average_predicted_volume": np.mean(pred1),
        "rsme": root_mean_squared_error(data1_val['product'], pred1)
    },

    "data2": {
        "average_predicted_volume": np.mean(pred2),
        "rsme": root_mean_squared_error(data2_val['product'], pred2)
    }
}

results_summary

{'data0': {'average_predicted_volume': np.float64(92.47214029036599),
  'rsme': 37.596556043309086},
 'data1': {'average_predicted_volume': np.float64(69.36937867411355),
  'rsme': 0.8941004282530967},
 'data2': {'average_predicted_volume': np.float64(94.909684188557),
  'rsme': 40.11354754639584}}

Ive found the average volumes and predicted reserves for the 3 models and that has offered us some insight. data0 shows moderate error at 36.63 rsme, data1 shows an excellent performance at .89 rsme, and finally data2 shows the worst rsme at 40.15. linear works great on data 1 however the rest dont perform as well. each region from data 0, 1, and 2 has a average predicted volume of 92.40, 68.14, 94.77 respectively and offer insight to 'typical product' values. It might be a good idea to work data0 and data 2 on a more complex model but for now this is what we will work with.

## Profit Calculation

In [23]:
reserves_needed = 100000000/4500
reserves_needed

22222.222222222223

In [24]:
reserves_per_well = reserves_needed / 200
reserves_per_well

111.11111111111111

This is the target at least 111.11 (thousand) avg reserves to be profitable

In [14]:
# Get top 200 indices from predictions
top_200_idx0 = np.argsort(pred0.astype(float))[-200:][::-1]
top_200_idx1 = np.argsort(pred1.astype(float))[-200:][::-1]
top_200_idx2 = np.argsort(pred2.astype(float))[-200:][::-1]


In [15]:
# Get top 200 predictions and corresponding true values
top_200_pred0 = pred0[top_200_idx0]
top_200_true0 = data0_val['product'].values[top_200_idx0]

top_200_pred1 = pred1[top_200_idx1]
top_200_true1 = data1_val['product'].values[top_200_idx1]

top_200_pred2 = pred2[top_200_idx2]
top_200_true2 = data2_val['product'].values[top_200_idx2]

Sorted and got the top 200 candidates for calculations

In [16]:
# ravel data for rmse
top_200_true0 = top_200_true0.ravel()
top_200_pred0 = top_200_pred0.ravel()
top_200_true1 = top_200_true1.ravel()
top_200_pred1 = top_200_pred1.ravel()
top_200_true2 = top_200_true2.ravel()
top_200_pred2 = top_200_pred2.ravel()
# Compute averages and RMSE
average_top_200_pred0 = np.mean(top_200_pred0)
average_top_200_pred1 = np.mean(top_200_pred1)
average_top_200_pred2 = np.mean(top_200_pred2)

rmse_top_200_pred0 = root_mean_squared_error(top_200_true0, top_200_pred0)
rmse_top_200_pred1 = root_mean_squared_error(top_200_true1, top_200_pred1)
rmse_top_200_pred2 = root_mean_squared_error(top_200_true2, top_200_pred2)

In [17]:
average_top_200_pred0


np.float64(70.75203626074905)

In [18]:
average_top_200_pred1

np.float64(111.2306603008724)

In [19]:
average_top_200_pred2

np.float64(85.91710895769742)

In [20]:
rmse_top_200_pred0

26.356966105388775

In [21]:
rmse_top_200_pred1

0.2385135915064325

In [22]:
rmse_top_200_pred2

14.292412845292148

Ive taken to top 200 candidates as we would only consider these in the final evaluation anyway and found their averages and rmse for 0,1, and 2. In the 0 dataset we see 70.75 (thousand) barrels of oil which is under our target and our rmse is at 26% as well not the greatest option. data 1 continues to look very promising with a 111.23 barrels meaning it is over our target and with a rmse of .23% making it an extremelly reliable model. data 2 displays 85 barrels under our target and has a 14% rmse alos making it less reliable.

In [32]:
def bootstrap_profit(predictions, true_values, n_iterations=1000, budget_million=100, num_wells=200, revenue_per_unit=4500):

    predictions = predictions.ravel()
    true_values = true_values.ravel()
    n_samples = len(predictions)

    cost_per_well = (budget_million * 1_000_000) / num_wells
    profits = []

    for _ in range(n_iterations):
        indices = np.random.choice(n_samples, n_samples, replace=True)
        sample_preds = predictions[indices]

        avg_volume = sample_preds.mean()
        revenue = avg_volume * revenue_per_unit
        profit = revenue - cost_per_well
        profits.append(profit)

    profits = np.array(profits)
    mean_profit = np.mean(profits)
    conf_interval = np.percentile(profits, [2.5, 97.5])
    loss_risk_percent = np.mean(profits < 0) * 100

    return mean_profit, conf_interval, loss_risk_percent


In [33]:
result_data0 = bootstrap_profit(top_200_pred0, top_200_true0)
result_data0

(np.float64(-181615.83682662935),
 array([-181615.83682663, -181615.83682663]),
 np.float64(100.0))

In [34]:
result_data1 = bootstrap_profit(top_200_pred1, top_200_true1)
result_data1

(np.float64(537.9713539258228),
 array([537.97135393, 537.97135393]),
 np.float64(0.0))

In [35]:
result_data2 = bootstrap_profit(top_200_pred2, top_200_true2)
result_data2

(np.float64(-113373.00969036158),
 array([-113373.00969036, -113373.00969036]),
 np.float64(100.0))

I compiled a few steps into this algorithm and ill list what we have here we re printed the avg volume and rmse to have a place that stores all the information in 1 function, we have calculated profit for each region (0 = -181615, 1 = 537, 2 = -113373) showing that our only profitable region is dataset 1. our risk is our bottom number data set 0 and 2 have a 100% risk and 1 has a 0%. a sa result of my findings i would suggest using dataset 1's region as the site for drillign as it is the only profitable site

## Conclusion

This project aimed to support strategic decision-making in oil well development through data-driven analysis. By leveraging machine learning predictions of oil reserve volumes, combined with financial modeling and risk assessment, we evaluated multiple regions to identify the most promising candidates for investment.

Using a combination of RMSE-based risk filtering and bootstrapping for profit estimation, we were able to estimate not only expected returns but also the likelihood of financial loss. This allowed us to recommend the region with the most favorable balance of expected profit and low risk, ensuring smarter capital allocation and improved confidence in development outcomes.